In [1]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
pitch_velo = pd.read_csv('metadata.csv')
force_plate = pd.read_csv('energy_flow.csv')
force_plate = force_plate.dropna()
force_plate[:20]

,session_pitch,time,rear_knee_energy_transfer_stp,rear_knee_energy_transfer_jfp,rear_knee_energy_generated,lead_knee_energy_transfer_stp,lead_knee_energy_transfer_jfp,lead_knee_energy_generated,rear_hip_energy_transfer_stp,rear_hip_energy_transfer_jfp,...,thorax_dist_seg_pwr,thorax_prox_seg_pwr,rear_thigh_dist_seg_pwr,rear_thigh_prox_seg_pwr,pkh_time,fp_10_time,fp_100_time,MER_time,BR_time,MIR_time
2,1031_2,0.0056,45.3350,20.1973,-8.0818,17.7538,6.0121,44.9851,0.1649,19.1142,...,1.3430,94.0265,-127.0096,65.5323,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
3,1031_2,0.0083,38.2959,19.3669,-8.7513,14.0252,4.4715,34.3861,0.0000,18.4929,...,1.5969,73.0565,-104.4922,57.6628,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
4,1031_2,0.0111,31.5861,18.5402,-9.5193,10.4432,3.0432,24.3333,0.0000,17.8197,...,1.4174,53.4393,-82.9323,50.1263,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
5,1031_2,0.0139,25.5195,17.7039,-10.4014,7.1676,1.7857,15.2168,0.0000,17.0917,...,0.8886,35.8775,-63.2113,43.2235,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
6,1031_2,0.0167,20.3575,16.8466,-11.4175,4.3307,0.7447,7.3494,0.0000,16.3050,...,0.1042,20.9140,-46.0584,37.2042,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
7,1031_2,0.0194,16.2879,15.9611,-12.5906,2.0265,-0.0497,0.9439,0.0000,15.4554,...,-0.8418,8.8945,-31.9950,32.2489,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
8,1031_2,0.0222,13.4109,15.0464,-13.9439,0.0000,-0.5844,-3.8984,0.0000,14.5387,...,-1.8623,-0.0501,-21.3023,28.4573,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
9,1031_2,0.0250,11.7354,14.1094,-15.4987,-0.8373,-0.8633,-7.1876,0.0000,13.5523,...,-2.8824,-5.9995,-14.0143,25.8448,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
10,1031_2,0.0278,11.1831,13.1642,-17.2711,-1.4393,-0.9046,-9.0342,-2.5615,12.4951,...,-3.8412,-9.2211,-9.9336,24.3473,0.625,1.1806,1.1917,1.3056,1.3389,1.3778
11,1031_2,0.0306,11.6014,12.2316,-19.2705,-1.5833,-0.7381,-9.6299,-2.6995,11.3686,...,-4.6905,-10.1304,-8.6690,23.8330,0.625,1.1806,1.1917,1.3056,1.3389,1.3778


In [3]:
del pitch_velo['filename_new']
del pitch_velo['modelname_new'] #commented out once complete
pitch_velo

,user,session,session_pitch,session_mass_kg,session_height_m,age_yrs,playing_level,pitch_speed_mph
0,750,1031,1031_2,110.6784,1.9304,21.40,college,90.4
1,750,1031,1031_3,110.6784,1.9304,21.40,college,90.4
2,774,1097,1097_1,101.1528,1.9304,19.41,college,77.6
3,774,1097,1097_2,101.1528,1.9304,19.41,college,77.0
4,774,1097,1097_3,101.1528,1.9304,19.41,college,76.1
...,...,...,...,...,...,...,...,...
406,870,3252,3252_1,88.4520,1.8796,20.38,college,80.8
407,870,3252,3252_2,88.4520,1.8796,20.38,college,80.8
408,870,3252,3252_3,88.4520,1.8796,20.38,college,80.8
409,870,3252,3252_4,88.4520,1.8796,20.38,college,83.3


In [4]:
ball_release = force_plate[force_plate['time']== force_plate['BR_time']]
ball_release.head()
##gets ball release frames from force plate data

,session_pitch,time,rear_knee_energy_transfer_stp,rear_knee_energy_transfer_jfp,rear_knee_energy_generated,lead_knee_energy_transfer_stp,lead_knee_energy_transfer_jfp,lead_knee_energy_generated,rear_hip_energy_transfer_stp,rear_hip_energy_transfer_jfp,...,thorax_dist_seg_pwr,thorax_prox_seg_pwr,rear_thigh_dist_seg_pwr,rear_thigh_prox_seg_pwr,pkh_time,fp_10_time,fp_100_time,MER_time,BR_time,MIR_time
482,1031_2,1.3389,-529.6774,1045.7377,-411.0764,0.0,986.7805,1803.8335,0.0000,-707.0322,...,2736.1917,-1791.9441,-5.0074,104.9839,0.6250,1.1806,1.1917,1.3056,1.3389,1.3778
847,1031_3,0.8250,-563.0821,1149.2876,-513.8955,0.0,747.2073,1382.6419,0.0000,-662.1831,...,2458.5579,-1770.0225,10.5157,72.3100,0.0917,0.6639,0.6806,0.7917,0.8250,0.8611
1278,1097_1,1.0028,-50.1856,-85.2700,61.9311,0.0,590.3287,-276.9181,-105.7167,-1053.7740,...,1441.3354,-1680.0410,1159.4906,-135.4557,0.1056,0.7722,0.8278,0.9722,1.0028,1.0500
1730,1097_2,1.0611,274.8673,13.4430,171.8570,0.0,1209.1444,399.2850,0.0000,-1085.4525,...,1077.9598,-3193.7871,49.5306,460.1674,0.1722,0.8528,0.8917,1.0306,1.0611,1.1083
2165,1097_3,1.0306,78.2917,-202.7941,-65.5761,0.0,549.1521,-559.9877,0.0000,-1180.8433,...,1263.8380,-1698.6222,1165.5778,-124.5025,0.1472,0.8361,0.8472,1.0028,1.0306,1.0778


In [5]:
pitch_velo['mph_bin'] = pd.qcut(pitch_velo['pitch_speed_mph'], 
                                q=3, labels = ['bad', 'average', 'elite'])
pitch_velo[:10]

,user,session,session_pitch,session_mass_kg,session_height_m,age_yrs,playing_level,pitch_speed_mph,mph_bin
0,750,1031,1031_2,110.6784,1.9304,21.40,college,90.4,elite
1,750,1031,1031_3,110.6784,1.9304,21.40,college,90.4,elite
2,774,1097,1097_1,101.1528,1.9304,19.41,college,77.6,bad
3,774,1097,1097_2,101.1528,1.9304,19.41,college,77.0,bad
4,774,1097,1097_3,101.1528,1.9304,19.41,college,76.1,bad
5,653,1170,1170_1,82.1016,1.9304,20.52,college,80.9,bad
6,653,1170,1170_2,82.1016,1.9304,20.52,college,82.6,bad
7,738,1250,1250_1,78.4728,1.7526,21.49,college,88.1,elite
8,738,1250,1250_3,78.4728,1.7526,21.49,college,88.6,elite
9,789,1313,1313_1,84.3696,1.8288,19.02,college,85.8,average


In [6]:
label_bin = LabelEncoder()

In [7]:
pitch_velo['mph_bin'] = label_bin.fit_transform(pitch_velo['mph_bin'])
pitch_velo['mph_bin'].value_counts()

1    138
0    137
2    136
Name: mph_bin, dtype: int64

In [8]:
del ball_release['time']

In [9]:
len(ball_release)
len(pitch_velo)

411

In [10]:
df = pd.merge(pitch_velo, ball_release, how='inner', left_on='session_pitch', right_on='session_pitch')
df.head()

,user,session,session_pitch,session_mass_kg,session_height_m,age_yrs,playing_level,pitch_speed_mph,mph_bin,rear_knee_energy_transfer_stp,...,thorax_dist_seg_pwr,thorax_prox_seg_pwr,rear_thigh_dist_seg_pwr,rear_thigh_prox_seg_pwr,pkh_time,fp_10_time,fp_100_time,MER_time,BR_time,MIR_time
0,750,1031,1031_2,110.6784,1.9304,21.40,college,90.4,2,-529.6774,...,2736.1917,-1791.9441,-5.0074,104.9839,0.6250,1.1806,1.1917,1.3056,1.3389,1.3778
1,750,1031,1031_3,110.6784,1.9304,21.40,college,90.4,2,-563.0821,...,2458.5579,-1770.0225,10.5157,72.3100,0.0917,0.6639,0.6806,0.7917,0.8250,0.8611
2,774,1097,1097_1,101.1528,1.9304,19.41,college,77.6,1,-50.1856,...,1441.3354,-1680.0410,1159.4906,-135.4557,0.1056,0.7722,0.8278,0.9722,1.0028,1.0500
3,774,1097,1097_2,101.1528,1.9304,19.41,college,77.0,1,274.8673,...,1077.9598,-3193.7871,49.5306,460.1674,0.1722,0.8528,0.8917,1.0306,1.0611,1.1083
4,774,1097,1097_3,101.1528,1.9304,19.41,college,76.1,1,78.2917,...,1263.8380,-1698.6222,1165.5778,-124.5025,0.1472,0.8361,0.8472,1.0028,1.0306,1.0778


In [11]:
#creating test and training splits to train and test the model
X = df.drop(labels = ['mph_bin', 'pitch_speed_mph', 'user', 'playing_level', 
                      'age_yrs', 'session', 'session_pitch'], axis = 1)
y = df['mph_bin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 12)

In [12]:
#scale values
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X_train, y_train)
predict_rfc = rfc.predict(X_test)

In [14]:
print(classification_report(y_test, predict_rfc))
print(confusion_matrix(y_test, predict_rfc))
##67, 73, 

              precision    recall  f1-score   support

           0       0.70      0.79      0.75        24
           1       0.96      0.77      0.85        30
           2       0.77      0.85      0.81        27

    accuracy                           0.80        81
   macro avg       0.81      0.80      0.80        81
weighted avg       0.82      0.80      0.81        81

[[19  1  4]
 [ 4 23  3]
 [ 4  0 23]]


In [15]:
clf = svm.SVC()
clf.fit(X_train, y_train)
predict_clf = clf.predict(X_test)

In [16]:
print(classification_report(y_test, predict_clf))
print(confusion_matrix(y_test, predict_clf))
#consistently 50%

              precision    recall  f1-score   support

           0       0.62      0.75      0.68        24
           1       0.92      0.77      0.84        30
           2       0.74      0.74      0.74        27

    accuracy                           0.75        81
   macro avg       0.76      0.75      0.75        81
weighted avg       0.77      0.75      0.76        81

[[18  2  4]
 [ 4 23  3]
 [ 7  0 20]]
